# LangGraph 中的記憶體最佳化與管理
## 為什麼需要記憶體管理?

當我們建構具備記憶能力的 Agent 時，最容易忽略的就是記憶體管理的重要性。首先是 LLM 的上下文限制問題，不同模型有不同的 token 上限，例如：
- GPT-4 從 8k 到 128k tokens
- Claude 達 200k tokens

但問題在於：Agent 是持續運行的。
當對話與內部狀態不斷累積，很快就會觸及模型的 token 天花板，導致：
- 對話被強制截斷
- 模型注意力被耗散
- token 消耗大
- 效能降低


## 記憶類型與生命週期概覽

要有效管理 Agent 的記憶，第一步是理解：
不是所有記憶都應該被同樣對待。

我們可以依「用途」與「生命週期」將記憶劃分為不同類型。
### 1. 短期對話記憶（Short-term Conversation Memory）
- 用途：維持當前對話的上下文連貫性
- 生命週期：僅限於單一會話
- 常見策略：
    - 只保留最近幾輪
    - 或最近 N 個 tokens

這類記憶不需要永久保存，過期即丟是正常設計。

### 2. 任務與目標記憶（Task / Goal Memory）
- 記錄內容：
    - 當前任務目標
    - 已完成步驟
    - 待辦事項（TODO）

- 特性：與特定任務綁定

當任務完成後，這類記憶可以：
- 被歸檔
- 或直接清除

不需要長期佔用「工作記憶」。

### 3. 長期使用者與知識記憶（Long-term Memory）
這一類包括：
- 使用者偏好
- 累積的背景知識
- 系統運行經驗

理論上它們需要長期保存，但實務上仍必須引入衰減機制，因為：

- 偏好會改變
- 知識會過時
- 長期未使用的記憶，價值會下降

「永久保存」不等於「永久活躍」。

## Active Memory 與 Archived Memory 的分層設計

在實際系統中，一個常見且有效的做法是將記憶分成兩個層級。
### 1. Active Memory（主動記憶）
Active Memory 是系統中的熱資料（Hot Data）層,存放那些頻繁被存取、對當前任務至關重要的記憶。這類記憶通常包括最近的對話歷史、正在進行的任務狀態,以及使用者最常用到的偏好設定。由於需要在每次 Agent 推理時快速讀取，Active Memory 會保存在高效能的儲存系統中，例如記憶體資料庫或向量資料庫的高速索引區。然而，高效能意味著高成本，因此 Active Memory 的容量必須受到嚴格控制，通常只保留數千到數萬條記錄。系統會透過重要性評分、存取頻率和時效性等指標，持續評估哪些記憶應該留在這一層，一旦記憶的價值下降或長時間未被使用，就會被降級到 Archived Memory，為更重要的新記憶騰出空間。

### Archived Memory（封存記憶）
Archived Memory 是系統的冷資料（Cold Data）層，用於長期保存那些不常使用但未來可能還需要的記憶。這些記憶可能是幾週前的對話內容、已完成任務的歷史記錄，或是很少被觸發的使用者偏好。由於存取頻率低，Archived Memory 可以使用成本較低的儲存方案，例如物件儲存或壓縮後的資料庫，檢索速度雖然較慢但容量幾乎不受限制。重要的是,歸檔並不等於刪除,當 Agent 在 Active Memory 中找不到相關資訊時，仍然可以向 Archived Memory 發起查詢，如果發現某條歸檔記憶突然變得重要(例如使用者提到很久以前的話題)，系統可以將其重新啟動並提升回 Active Memory。這種動態的分層機制讓 Agent 既能保持快速反應，又能在需要時調用深層的歷史記憶，就像人類大腦在日常思考時主要依賴短期記憶，但在特定情境下也能喚醒塵封已久的往事。

## 對話上下文壓縮與裁切
### Trimming：限制對話歷史長度

**固定窗口：**

In [ ]:
def fixed_window_trim(messages, max_count=10):
    """保留最近 N 條訊息"""
    return messages[-max_count:]

**滑動窗口：**

In [ ]:
def sliding_window_trim(messages, max_tokens=4000):
    """基於 token 數量的滑動窗口"""
    total_tokens = 0
    trimmed = []
    
    for msg in reversed(messages):
        msg_tokens = count_tokens(msg)
        if total_tokens + msg_tokens > max_tokens:
            break
        trimmed.insert(0, msg)
        total_tokens += msg_tokens
    
    return trimmed

**在 LangGraph 中的 reducer 實作**

In [1]:
from langgraph.graph import MessagesState
from typing import Annotated

def trim_messages_reducer(existing, new):
    """自動裁切訊息的 reducer"""
    all_messages = existing + new
    return sliding_window_trim(all_messages, max_tokens=4000)

class TrimmedState(MessagesState):
    messages: Annotated[list, trim_messages_reducer]

### to be continued